In [27]:
import polars as pl
import polars.selectors as cs
from utils import PathsData, import_data, split_data, plot_correlation
from features import FeaturesFrame
from models import Model, ModelEnum, ParamGridEnum

# import matplotlib.pyplot as plt
# import seaborn as sns
# from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
# from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler
from xgboost import XGBRegressor


x_validation = import_data(PathsData.X_TEST.value) #! to use in the end to make a submission
x = import_data(PathsData.X_TRAIN.value)
y = import_data(PathsData.Y_TRAIN.value)

x_train, x_test, y_train, y_test = split_data(x=x, y=y)

In [6]:
test_train = FeaturesFrame(x_train).encode_label() #.add_feature_square().scale_standard() #.add_feature_interactions() #.add_feature_square() #.scale_standard(set="train")
test_test = FeaturesFrame(x_test).encode_label() #.add_feature_square().scale_standard(set='test', train_scaler=scaler) #.add_feature_interactions() #.add_feature_square() #.scale_standard(set="test", train_scaler=scaler)

In [9]:
test_train = test_train.select('arret','encoded_gare')
test_test = test_test.select('arret', 'encoded_gare')

In [10]:
model = Model(model_enum=ModelEnum.XGBoost)
model_fit = model.fit(test_train.to_numpy(), y_train=y_train.to_numpy())
model.evaluate(X_test=test_test.to_numpy(), y_test=y_test.to_numpy())

Model Evaluation - MSE: 4.3097, R2: 0.0675, MAE: 0.8515


(4.30968257451896, 0.06750007177579564, 0.8515066898405728)

In [11]:
model = Model(model_enum=ModelEnum.XGBoost)
params_opti_mlp = model.grid_search(test_train.to_pandas(), y_train=y_train.to_pandas(), param_grid = ParamGridEnum.XGBoost.value)

Fitting 3 folds for each of 108 candidates, totalling 324 fits


108it [00:00, 5820.11it/s]


In [15]:
params_opti_xgb = {
        "n_estimators": 200,
        "learning_rate": 0.1,
        "max_depth":5,
        "subsample": 0.8,
        "colsample_bytree": 1
    }

In [25]:
mod = XGBRegressor().set_params(**params_opti_xgb)
mod_fit = mod.fit(test_train.to_numpy(),y_train.to_numpy())
preds = mod.predict(X=test_test.to_numpy())

In [28]:
mse = mean_squared_error(y_test, preds)
r2 = r2_score(y_test, preds)
mae = mean_absolute_error(y_test,preds)
print(f"Model Evaluation - MSE: {mse:.4f}, R2: {r2:.4f}, MAE: {mae:.4f}")

Model Evaluation - MSE: 4.2048, R2: 0.0902, MAE: 0.8183


In [16]:
model = Model(model_enum=ModelEnum.XGBoost, params=params_opti_xgb)
model_fit = model.fit(test_train.to_numpy(), y_train=y_train.to_numpy())
model.evaluate(X_test=test_test.to_numpy(), y_test=y_test.to_numpy())

TypeError: EnumType.__call__() got an unexpected keyword argument 'n_estimators'

In [ ]:
# model = Model(model_enum=ModelEnum.GradientBoosting)
# params_opti_gb = model.optimize_parameters(test_train.to_pandas(), y_train=y_train.to_pandas(), param_grid = ParamGridEnum.GradientBoosting.value)

In [ ]:
# model = Model(model_enum=ModelEnum.SVR)
# params_opti_svr = model.optimize_parameters(test_train.to_pandas(), y_train=y_train.to_pandas(), param_grid = ParamGridEnum.SVR.value)

In [ ]:
# model = Model(model_enum=ModelEnum.KNN)
# params_opti_knn = model.optimize_parameters(test_train.to_pandas(), y_train=y_train.to_pandas(), param_grid = ParamGridEnum.KNN.value)